# The Graph Data Access

In this notebook, we retrieve live Rai system data from [The Graph](https://thegraph.com/explorer/subgraph/reflexer-labs/rai-mainnet?query=Example%20query) and save it as a CSV for analysis in the [Graph Analysis](TheGraphDataAnalysis.ipynb) notebook.

## Full Reflexer Labs dashboad query as obtained from Reflexer team
```
{
  internalCoinBalances(where: {accountHandler: "${GEB_STABILITY_FEE_TREASURY}"}) { 
    balance
  }
  accountingEngine(id: "current") { 
    activeDebtAuctions
    activeSurplusAuctions
  }
  collateralType(id: "ETH-A") {
    currentPrice {
      value
    }
    currentMedianizerUpdate {
      value
    }
    debtCeiling
    activeLiquidations
    liquidationsStarted
    totalCollateral
    totalAnnualizedStabilityFee
  }
  systemState(id: "current") { 
    coinUniswapPair {
      reserve0
      reserve1
    }
    currentCoinMedianizerUpdate{
      value
    }
    currentRedemptionRate {
      eightHourlyRate
      annualizedRate
      hourlyRate
      createdAt
    }
    currentRedemptionPrice {
      value
    }
    erc20CoinTotalSupply
    globalDebt
    globalDebtCeiling
    totalActiveSafeCount
    coinAddress
    wethAddress
    systemSurplus
    debtAvailableToSettle
  }
  dailyStats(first: 1000, orderDirection: desc) {
    globalDebt
    timestamp
    redemptionRate {
      twentyFourHourlyRate
    }
  }
  hourlyStats(first: 1000, orderBy: timestamp, orderDirection: desc) {
    globalDebt
    timestamp
    redemptionRate {
      hourlyRate
    }
    redemptionPrice{
      value
    }
    marketPriceUsd
  }
}
```

In [9]:
# import libraries
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt

# Graph
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

## Hourly stats

In [10]:
query_header = '''
query {{
    hourlyStats(first: 1000, skip:{}) {{'''

query_tail = '''    
}
}'''

query_body = '''
 timestamp
    blockNumber
    redemptionRate {
      hourlyRate
    }
    redemptionPrice {
      id
      block
      value
      
    }
    marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
    marketPriceEth # Price of COIN in ETH (uni pool price)
    globalDebt
    erc20CoinTotalSupply
'''


n = 0
hourly = []
while True:
    print(f'request {n+1}')
    query = query_header.format(n*1000) + query_body + query_tail
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats']
    print(f'results {len(s)}')
    hourly.extend(s)
    print(f'total {len(hourly)}')
    n += 1
    if len(s) < 1000:
        break
        
hourlyStats = pd.DataFrame(hourly)

request 1
results 1000
total 1000
request 2
results 238
total 1238


In [11]:
hourlyStats

,blockNumber,erc20CoinTotalSupply,globalDebt,marketPriceEth,marketPriceUsd,redemptionPrice,redemptionRate,timestamp
0,11857308,1988.440432971667032725,1988.577499489843641169436691184972,0.002589462210048543650949087456484599,4.670500166535429740591356399938391,"{'block': '11849022', 'id': '0xe313271188492c4...",{'hourlyRate': '1'},1613338681
1,11860793,2788.440432971667032725,2788.617717345781163523077653358556,0.002481355746535171105569862943787003,4.377049344386119132419461918241039,"{'block': '11860755', 'id': '0x40066774f6fb7d3...",{'hourlyRate': '1'},1613384616
2,11861040,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.406833024856393396658172078078713,"{'block': '11861008', 'id': '0x0d551e2a2d55fa8...",{'hourlyRate': '1'},1613387890
3,11861337,2788.440432971667032725,2788.657935201718685876567866781341,0.002481355746535171105569862943787003,4.419265710853707004204445293455575,"{'block': '11861237', 'id': '0x5c67dbb323db039...",{'hourlyRate': '1'},1613392101
4,11861474,3048.440432971667032725,3048.645954953510904626288422086589,0.001768934955701819449591545208406114,3.194329891923853932595347583210722,"{'block': '11861237', 'id': '0x5c67dbb323db039...",{'hourlyRate': '1'},1613394096
...,...,...,...,...,...,...,...,...
1233,12204611,33594316.17315388687412851,33841659.45244358661077294851817894,0.001481100046417661646680546800917116,3.077650071657512553327587373446738,"{'block': '12204611', 'id': '0xc982669263a637d...",{'hourlyRate': '0.999978938762814013832122833'},1617957529
1234,12204739,33594316.17315388687412851,33841237.01048489334195170725296134,0.001481100046417661646680546800917116,3.077650071657512553327587373446738,"{'block': '12204611', 'id': '0xc982669263a637d...",{'hourlyRate': '0.999978938762814013832122833'},1617959058
1235,12205014,33835816.17315388687412851,34082817.62689425387534166286292176,0.001480890804043685960248419879786498,3.06642135230109877044799310827871,"{'block': '12204766', 'id': '0x84b5d343286fbd6...",{'hourlyRate': '0.999978938762814013832122833'},1617962942
1236,12205409,33944413.793281159532545611,34191346.56742297913988350267736426,0.001480873550435953124324974899280158,3.090094411488190305935195373080927,"{'block': '12205288', 'id': '0x62b07b854aeb5dd...",{'hourlyRate': '0.999978938762814013832122833'},1617968280


In [12]:
hourlyStats.redemptionPrice.values[-1]

{'block': '12205772',
 'id': '0x2905220cd2adec7a741d59012eea2b1fbbc02d358ffe94792e9f7d3baee39434-33',
 'value': '3.027815530420218716970544302'}

In [13]:
hourlyStats.redemptionRate.values[-1]

{'hourlyRate': '0.999978293905335257900684155'}

In [14]:
hourlyStats['hourlyredemptionPriceActual'] = hourlyStats.redemptionPrice.apply(lambda x: x['value'])
hourlyStats['hourlyredemptionRateActual'] = hourlyStats.redemptionRate.apply(lambda x: x['hourlyRate'])
del hourlyStats['redemptionRate']
del hourlyStats['redemptionPrice']
hourlyStats['erc20CoinTotalSupply'] = hourlyStats['erc20CoinTotalSupply'].astype(float)
hourlyStats['globalDebt'] = hourlyStats['globalDebt'].astype(float)
hourlyStats['blockNumber'] = hourlyStats['blockNumber'].astype(int)
hourlyStats['hourlymarketPriceEth'] = hourlyStats['marketPriceEth'].astype(float)
del hourlyStats['marketPriceEth']
hourlyStats['hourlymarketPriceUsd'] = hourlyStats['marketPriceUsd'].astype(float)
del hourlyStats['marketPriceUsd']
hourlyStats['hourlyredemptionPriceActual'] = hourlyStats['hourlyredemptionPriceActual'].astype(float)
hourlyStats['hourlyredemptionRateActual'] = hourlyStats['hourlyredemptionRateActual'].astype(float)

In [15]:
hourlyStats.describe()

,blockNumber,erc20CoinTotalSupply,globalDebt,hourlyredemptionPriceActual,hourlyredemptionRateActual,hourlymarketPriceEth,hourlymarketPriceUsd
count,1.238000e+03,1.238000e+03,1.238000e+03,1238.000000,1238.000000,1238.000000,1238.000000
mean,1.203147e+07,3.473195e+07,3.511759e+07,3.054531,0.999970,0.001798,3.155879
std,9.978664e+04,1.211140e+07,1.219603e+07,0.043963,0.000055,0.000288,0.381351
min,1.185731e+07,1.988440e+03,1.988577e+03,3.013686,0.999796,0.001452,2.850455
25%,1.194500e+07,3.309371e+07,3.373953e+07,3.020102,0.999942,0.001645,2.995186
50%,1.202997e+07,3.592230e+07,3.632733e+07,3.031552,0.999992,0.001728,3.069590
75%,1.211847e+07,4.275394e+07,4.302486e+07,3.095314,1.000009,0.001937,3.215063
max,1.220577e+07,4.899876e+07,4.918825e+07,3.140000,1.000051,0.004474,8.055382


## Daily stats - WIP

## System State 

In [16]:
blocknumbers = hourlyStats.blockNumber.values.tolist()

state = []
for i in blocknumbers:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          reserve0
          reserve1
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState.head()
    

,coinAddress,coinUniswapPair,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,wethAddress
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '661.764842786150958141', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.522357955470745187436691184972,115792089237316195423570985008687.9,0.06853325908830422221834559248594176,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


In [17]:
systemState['blockNumber'] = blocknumbers

In [18]:
systemState.coinUniswapPair.values[-1]

{'reserve0': '32243864.854176256963147613',
 'reserve1': '47749.758432648976193814'}

In [19]:
systemState.currentCoinMedianizerUpdate.values[-1]

{'value': '3.088110892031286427'}

In [20]:
systemState.currentRedemptionRate.values[-1]

{'annualizedRate': '0.826837205369189259013964111',
 'createdAt': '1617973066',
 'eightHourlyRate': '0.999826364434436647587707258',
 'hourlyRate': '0.999978293905335257900684155'}

In [21]:
systemState.currentRedemptionPrice.values[-1]

{'value': '3.027815530420218716970544302'}

In [22]:
systemState.columns

Index(['coinAddress', 'coinUniswapPair', 'currentCoinMedianizerUpdate',
       'currentRedemptionPrice', 'currentRedemptionRate',
       'debtAvailableToSettle', 'erc20CoinTotalSupply', 'globalDebt',
       'globalDebtCeiling', 'systemSurplus', 'totalActiveSafeCount',
       'wethAddress', 'blockNumber'],
      dtype='object')

In [23]:
systemState['RedemptionRateAnnualizedRate'] = systemState.currentRedemptionRate.apply(lambda x: x['annualizedRate'])
systemState['RedemptionRateHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['hourlyRate'])
systemState['RedemptionRateEightHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['eightHourlyRate'])
systemState['RedemptionPrice'] = systemState.currentRedemptionPrice.apply(lambda x: x['value'])
systemState['RAIInUniswapV2(RAI/ETH)'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
del systemState['currentRedemptionRate']
del systemState['currentRedemptionPrice']
systemState['RedemptionRateAnnualizedRate'] = systemState['RedemptionRateAnnualizedRate'].astype(float)
systemState['RedemptionRateHourlyRate'] = systemState['RedemptionRateHourlyRate'].astype(float)
systemState['RedemptionRateEightHourlyRate'] = systemState['RedemptionRateEightHourlyRate'].astype(float)
systemState['RedemptionPrice'] = systemState['RedemptionPrice'].astype(float)
systemState['RAIInUniswapV2(RAI/ETH)'] = systemState['RAIInUniswapV2(RAI/ETH)'].astype(float)



In [24]:
systemState.head()

,coinAddress,coinUniswapPair,currentCoinMedianizerUpdate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,wethAddress,blockNumber,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,RAIInUniswapV2(RAI/ETH)
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '661.764842786150958141', 'reserv...",None,0,2788.440432971667032725,2788.522357955470745187436691184972,115792089237316195423570985008687.9,0.06853325908830422221834559248594176,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11857308,1.0,1.0,1.0,3.14,661.764843
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11860793,1.0,1.0,1.0,3.14,676.933727
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861040,1.0,1.0,1.0,3.14,676.933727
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '676.933726709008306816', 'reserv...",None,0,2788.440432971667032725,2788.657935201718685876567866781341,115792089237316195423570985008687.9,0.1087511150258265757085590152709335,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861337,1.0,1.0,1.0,3.14,676.933727
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'reserve0': '801.933726709008306816', 'reserv...",{'value': '0'},0,3048.440432971667032725,3048.645954953510904626288422086589,115792089237316195423570985008687.9,0.1161617985274885650688366678947734,3,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11861474,1.0,1.0,1.0,3.14,801.933727


## Internal Coin Balances - WIP

## Accounting Engine - WIP

## Eth Price

In [25]:
eth_price = []
for i in blocknumbers:
    query = '''
    {
      collateralType(block: {number:%s}, id: "ETH-A") {
        currentFsmUpdate {
          value
        }
        currentMedianizerUpdate {
          value
        }
        debtCeiling
        activeLiquidations
        liquidationsStarted
        totalCollateral
        totalAnnualizedStabilityFee
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['collateralType']
    eth_price.append(s)
        
collateralType = pd.DataFrame(eth_price)

collateralType['blockNumber'] = blocknumbers
collateralType.head()
    

,activeLiquidations,currentFsmUpdate,currentMedianizerUpdate,debtCeiling,liquidationsStarted,totalAnnualizedStabilityFee,totalCollateral,blockNumber
0,0,{'value': '1803.65643044'},{'value': '1803.65643044'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11857308
1,0,{'value': '1803.65643044'},{'value': '1763.97493608'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11860793
2,0,{'value': '1763.97493608'},{'value': '1775.97792296'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11861040
3,0,{'value': '1763.97493608'},{'value': '1780.98836373'},10000000,0,1.019999999999999999972831879,14.892360473369843164,11861337
4,0,{'value': '1763.97493608'},{'value': '1805.79273513'},10000000,0,1.019999999999999999972831879,16.892360473369843164,11861474


In [26]:
collateralType['Eth_price'] = collateralType.currentFsmUpdate.apply(lambda x: x['value'])
collateralType['Eth_price'] = collateralType['Eth_price'].astype(float)

In [27]:
hourlyStats = hourlyStats.merge(collateralType,how='inner',on='blockNumber')

# remove duplicate values
del systemState['erc20CoinTotalSupply']
del systemState['globalDebt']

hourlyStats = hourlyStats.merge(systemState,how='inner',on='blockNumber')

In [28]:
hourlyStats.to_csv('saved_results/RaiLiveGraphData.csv')

## Safes

## Conclusion

Using The Graph, a lot of data about the Rai system can be obtained for analyzing the health of the system. With some data manipulation, these data streams could be intergrated into the Rai cadCAD model to turn it into a true decision support system.